In [8]:
import pandas as pd
import numpy as np
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
ws = WS("./data_ckip")
pos = POS("./data_ckip")
ner = NER("./data_ckip")

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/ckiptagger/model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/ckiptagger/model_pos.py:56: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/ckiptagger/model_ner.py:57: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will

In [9]:
def word_pos_sentence(word_sentence, pos_sentence):
    result = ''
    assert len(word_sentence) == len(pos_sentence)
    for word, pos in zip(word_sentence, pos_sentence):
        # Ignore whitespace
        if pos != 'WHITESPACE' and pos != "FW":
            result += '{}({}) '.format(word, pos)
    result = result[:-1]
    return result

In [11]:
# Split the data into training and testing sets
datas = {
    'ptt': pd.read_csv('../data/ptt_cleaned.csv'),
    'dcard': pd.read_csv('../data/dcard_cleaned.csv'),
    'news': pd.read_csv('../data/news_cleaned.csv'),
}

for name, data in datas.items():
    # substitude the nan value with empty string
    data['content_cleaned'] = data['content_cleaned'].replace(np.nan, '', regex=True)
    word_sentence_list = ws(
        data['content_cleaned'],
        sentence_segmentation=True, # To consider delimiters
        segment_delimiter_set={'?', '？', '!', '！', '。', ',','，', ';', ':', '、', ' '})
    pos_sentence_list_19 = pos(word_sentence_list)
    count = 0 
    for sentence in zip(word_sentence_list, pos_sentence_list_19):
        count += 1
        with open(f'../data/pos_tagged_data/{name}/{count}.txt', 'w') as f:
            f.write(word_pos_sentence(sentence[0], sentence[1]) + '\n')

In [7]:
# 生成 lst 檔案
for name, data in datas.items():
    # num is the number of rows
    num = len(data)
    with open(f'../data/pos_tagged_data/{name}.lst', 'w') as f:
        for i in range(1, num+1):
            input_filename = f"{i}.txt"
            output_filename = str(i)
            f.write(f"./{name}/{input_filename} {output_filename}\n")
